In [1]:
from ximg_to_ypdf_autoencoder import Ximg_to_Ypdf_Autoencoder
from denoising_util import *
# Get the directory of the currently running file
current_dir = os.getcwd()

# Construct the path to the utils directory relative to the current file's directory
utils_dir = os.path.abspath(os.path.join(current_dir, '..', 'ml_backbone'))

# Add the utils directory to the Python path
sys.path.append(utils_dir)
from utils import DataMilking_Nonfat, DataMilking
from utils import CustomScheduler

# Check if CUDA (GPU support) is available
if torch.cuda.is_available():
    print("GPU is available!")
    device = torch.device("cuda")
elif torch.backends.mps.is_available() and torch.backends.mps.is_built():
    device = torch.device("mps")
    print("MPS is available. Using GPU.")
else:
    device = torch.device("cpu")
    print("MPS is not available. Using CPU.")

MPS is available. Using GPU.


In [2]:
# Load Dataset and Feed to Dataloader
datapath = "/Users/jhirschm/Documents/MRCO/Data_Changed/Test"
data = DataMilking_Nonfat(root_dir=datapath, pulse_number=2)
# Calculate the lengths for each split
train_size = int(0.8 * len(data))
val_size = int(0.1 * len(data))
test_size = len(data) - train_size - val_size

# Perform the split
train_dataset, val_dataset, test_dataset = random_split(data, [train_size, val_size, test_size])

# Create data loaders
train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=32, shuffle=False)
test_dataloader = DataLoader(test_dataset, batch_size=32, shuffle=False)


In [3]:
# Define the model
encoder_layers = [
    nn.Conv2d(1, 16, kernel_size=3, stride=2, padding=1),
    nn.Conv2d(16, 32, kernel_size=3, stride=2, padding=1),
    nn.Conv2d(32, 64, kernel_size=3, stride=2, padding=1)
]

decoder_layers = [
    nn.ConvTranspose2d(64, 32, kernel_size=3, stride=2, padding=1, output_padding=1),
    nn.ConvTranspose2d(32, 16, kernel_size=3, stride=2, padding=1, output_padding=1),
    nn.ConvTranspose2d(16, 1, kernel_size=3, stride=2, padding=1, output_padding=1)
]

autoencoder = Ximg_to_Ypdf_Autoencoder(encoder_layers, decoder_layers)

# Define the loss function and optimizer
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(autoencoder.parameters(), lr=0.0001)
max_epochs = 20
scheduler = CustomScheduler(optimizer, patience=3, early_stop_patience = 6, cooldown=2, lr_reduction_factor=0.5, max_num_epochs = max_epochs, improvement_percentage=0.01)
model_save_dir = "/Users/jhirschm/Documents/MRCO/Data_Changed/Test"

In [4]:
if not os.path.exists(model_save_dir):
    os.makedirs(model_save_dir)
        

In [6]:
identifier = "testAutoencoder"
autoencoder.to(device)
best_model, best_epoch, train_losses, val_losses, best_val_loss= autoencoder.train_model(train_dataloader, val_dataloader, criterion, optimizer, scheduler, model_save_dir, identifier, device, checkpoints_enabled=True, resume_from_checkpoint=False, max_epochs=max_epochs)


Epoch [1/20] - Train Loss: 1.1065313834, Validation Loss: 0.9908687197


Improvement


/Users/jhirschm/Documents/MRCO/COOKIE_ML/src/ml_backbone/utils/custom_schedulers.py:42: UserWarning: Epoch parameter is deprecated and will be removed in a future release.
  warnings.warn("Epoch parameter is deprecated and will be removed in a future release.", UserWarning)


Epoch [2/20] - Train Loss: 0.9906093570, Validation Loss: 0.9904687668


Epoch [3/20] - Train Loss: 0.9904603718, Validation Loss: 0.9904172077


Epoch [4/20] - Train Loss: 0.9904311397, Validation Loss: 0.9904002478


Epoch [5/20] - Train Loss: 0.9904194362, Validation Loss: 0.9903940169


Epoch 5: reducing learning rate from 0.000100 to 0.000050
Epoch [6/20] - Train Loss: 0.9904171628, Validation Loss: 0.9903934793


Epoch [7/20] - Train Loss: 0.9904168568, Validation Loss: 0.9903932237


Epoch [8/20] - Train Loss: 0.9904168226, Validation Loss: 0.9903930414


Epoch [9/20] - Train Loss: 0.9904165047, Validation Loss: 0.9903929360


Epoch 9: reducing learning rate from 0.000050 to 0.000025
Epoch [10/20] - Train Loss: 0.9904163858, Validation Loss: 0.9903929050


Epoch [11/20] - Train Loss: 0.9904166761, Validation Loss: 0.9903928855


Epoch [12/20] - Train Loss: 0.9904168763, Validation Loss: 0.9903928592


Early stopping at epoch 12
Early stopping at epoch 12



In [ ]:
results_file = os.path.join(model_save_dir, f"{identifier}_results.txt")
with open(results_file, 'w') as f:
    f.write("Model Training Results\n")
    f.write("======================\n")
    f.write(f"Data Path: {datapath}\n")
    f.write(f"Model Save Directory: {model_save_dir}\n")
    f.write("\nModel Parameters and Hyperparameters\n")
    f.write("-----------------------------------\n")
    f.write(f"Patience: {scheduler.patience}\n")
    f.write(f"Cooldown: {scheduler.cooldown}\n")
    f.write(f"Learning Rate Reduction Factor: {scheduler.lr_reduction_factor}\n")
    f.write(f"Improvement Percentage: {scheduler.improvement_percentage}\n")
    f.write(f"Initial Learning Rate: {optimizer.param_groups[0]['lr']}\n")
    f.write("\nModel Architecture\n")
    f.write("------------------\n")
    f.write(f"Encoder Layers: {encoder_layers}\n")
    f.write(f"Decoder Layers: {decoder_layers}\n")
    f.write("\nAdditional Notes\n")
    f.write("----------------\n")
    f.write("First trial on S3DF for Denoising.\n")